# Оценка точности работы сервиса (Caila)


Для оценки точности работы сервиса загрузим датасет для проверки.

Испоьзовался датасет NERUS - https://github.com/natasha/nerus

Nerus — большой датасет русского языка , аннотированный тегами POS, синтаксическими деревьями и тегами NER (PER, LOC, ORG). Nerus имеет определенную степень ошибок в разметке, но качество довольно высокое. Корпус содержит ~700 тыс. новостных статей из Lenta.ru. Использовались инструменты из проекта Natasha: Razdel для сегментации предложений и токенов, модели Slovnet BERT для морфологии, синтаксиса и аннотации NER. Разметка хранится в стандартном формате CoNLL-U.

In [465]:
import requests
import json
import re
import time

In [466]:
from nerus import load_nerus

#Загружаем датасет из файла
docs = load_nerus(r"C:\Users\tkav1\Downloads\nerus_lenta.conllu.gz")

In [467]:
from nltk.stem.snowball import SnowballStemmer

#Берем подвыборку из датасета размера samples
def take_samples_from_dataset(samples):
    
    texts_persons = [] #список для сущностей с ФИО
    texts_list = [] #список для текстов
    
    for i in range(samples):
        
        list_pers = []
    
        #Проходимся по первым-i текстам
        doc = next(docs)
    
        text_full = doc.ner.text
        texts_list.append(text_full) #Добавляем текст в список
    
        #Ищем участки текста, помеченные 'PER' - указатель на ФИО
        for j in range(len(doc.ner.spans)):
            
            if doc.ner.spans[j].type == 'PER':
                person = doc.ner.text[doc.ner.spans[j].start:doc.ner.spans[j].stop]
                list_pers.append(person) #Если нашли указатель - добавляем сущность в список сущностей
                 
        texts_persons.append(list_pers) #Каждый текст имеет свой список сущностей
        
    return texts_list, texts_persons

In [468]:
#Берем подвыборку из 100 примеров, результат - два списка: (1) списки сущностей (2) список текстов
texts_list, texts_persons  = take_samples_from_dataset(100)

In [469]:
#Пример
texts_list[0], texts_persons[0]

('Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
 ['Татьяна Голикова', 'Голиковой'])

In [470]:
from nltk.stem.snowball import SnowballStemmer

def tetx_list_stemming(entity_list):
    
    stemmer = SnowballStemmer("russian")
    text_persons_stem = []
    
    for entity in entity_list:
        #print(entity)
        stemmed_name = []
        for name in entity:
            #print(name)
            words = name.split()
            stemmed_words = [stemmer.stem(word).capitalize() for word in words]
            stemmed_name.append(' '.join(stemmed_words))
            #print(stemmed_name)
            
        text_persons_stem.append(stemmed_name)
        
    return text_persons_stem

In [471]:
# Стеммизируем верные ответы
texts_persons_stem = tetxs_list_stemming(texts_persons)

In [472]:
#Пример
texts_persons_stem[0]

['Татья Голиков', 'Голиков']


Верные данные подготовили, теперь можно отправлять тексты с датасета на оценку на сам сервис.

В данном случае используется упрощенный код, непосредственно применяемый в сервисе. Код делает запрос к API CAILA для извлечения полных и неполных имен (ФИО) из текстов, переданных в виде списка предложений.

In [473]:
#Здесь задается URL для API CAILA и заголовки, включая Authorization с токеном доступа. Важно использовать свой API ключ для доступа
url = 'https://caila.io/api/adapters/openai/chat/completions'
headers = {
    'Authorization': 'API-KEY', #!!!ВВЕДИТЕ СВОЙ API_KEY с CAILA!!!
    'Content-Type': 'application/json'
}

#Эта функция принимает на вход список текстов data_input и возвращает список имен (ФИО), найденных в этих текстах
#data_input - входные данные, model_name - название модели, open_ai - относится ли к семейству opean_ai
def model_sample(data_input, model_name, open_ai: bool):

    names_list = []

    # Подсчитываем общее количество символов (для оценки времени работы)
    total_chars = sum(len(text) for text in data_input)

    # Замеряем время начала
    start_time = time.time()

    # Проходимся по каждому предложению в списке data_input. Каждое текст обрабатывается отдельно.
    for i in range(len(data_input)):
        
        # Данные в формате JSON
        data = {
            "texts": [data_input[i]]
        }

        # Преобразуем массив предложений в один текст для отправки в запросе
        text_input = " ".join(data["texts"])

        # Уточняем, относиться к openai 
        if open_ai == True:
            model_root = 'just-ai/openai-proxy/'
        else:
            model_root = 'just-ai/'
        
        # Формируется запрос к модели с инструкцией извлечь полные и неполные имена из текста.
        request_data = {
            "model": f"{model_root}{model_name}",
            "messages": [
                {"role": "user", "content": f"Верни из исходного текста все упоминания полных и неполных ФИО \
                (как полные фамилия, имя, отчество, так и их части: фамилия или имя, либо их сочетания).\
                Сохрани оригинальное написание ФИО, представленное в тексте, без изменений. \
                Если в тексте нет упоминаний ФИО, просто верни пустой результат. Исходный текст: {text_input}"}
            ]
        }
        
        # Запрос отправляется с помощью requests.post, и ответ преобразуется в JSON для дальнейшей обработки.
        response = requests.post(url, headers=headers, json=request_data)
        response_json = response.json()
        
        # Обрабатываем ответ
        try:
            #Из ответа API извлекаются данные с помощью метода .get(), чтобы избежать ошибок, если данных нет.
            choices = response_json.get('choices', [])
            if choices:
                content = choices[0].get('message', {}).get('content', '')
        
                # Регулярное выражение ищет:
                #Полные имена (две части с заглавными буквами для латиницы и кириллицы).
                #Отдельные имена (одна часть для неполных имен).
                #Имена сохраняются в переменной names.
                name_pattern = re.compile(r'\b([A-Z][a-z]+ [A-Z][a-z]+|[A-Z][a-z]+|[А-ЯЁ][а-яё]+ [А-ЯЁ][а-яё]+|[А-ЯЁ][а-яё]+)\b')
                names = name_pattern.findall(content)
            else:
                print("Нет данных в ответе.")
        except Exception as e:
            #Ловим возможные ошибки при обработке ответа от API.
            print(f"Ошибка при обработке ответа: {e}")
            
        #Найденные имена добавляются в список names_list
        names_list.append(names)
        
    # Замеряем время окончания
    end_time = time.time()

    # Вычисляем время выполнения
    elapsed_time = end_time - start_time

    # Вычисляем скорость обработки в символах в секунду
    chars_per_second = total_chars / elapsed_time if elapsed_time > 0 else 0
        
    return names_list, elapsed_time, chars_per_second

### Смотрим 3 модели: gpt4-mini, gpt3_5-turbo и vllm-llama3.1-70b-4q

In [474]:
#Предсказываем/ищем сущности + время работы + число обработанных символов в секунду
names_gpt4m, elapsed_time_gpt4m, chars_sec_gpt4m = model_sample(texts_list, 'gpt-4o-mini', True) #gpt-4o-mini
names_gpt3_5t, elapsed_time_gpt3_5t, chars_sec_gpt3_5t = model_sample(texts_list, 'gpt-3.5-turbo', True) #gpt-3.5-turbo
names_llama, elapsed_time_llama, chars_sec_llama = model_sample(texts_list, 'vllm-llama3.1-70b-4q', False) #vllm-llama3.1-70b-4q

#names_gpt3_5t_16k, elapsed_time_gpt3_5t_16k, chars_sec_gpt3_5t_16k = model_sample(texts_list, 'gpt-3.5-turbo-16k') #gpt-3.5-turbo-16k

In [475]:
#Применяем стемминг для предсказанных ответов и создаем под них отдельные списки
names_gpt4m_stem = tetx_list_stemming(names_gpt4m) #Результат для gpt-4o-mini после стемминга
names_gpt3_5t_stem = tetx_list_stemming(names_gpt3_5t) #Результат для gpt-3.5-turbo после стемминга
names_llama_stem = tetx_list_stemming(names_llama) #Результат для vllm-llama3.1-70b-4q после стемминга

## Время работы и количество обработанных символов в секунду

In [476]:
def time_tests(model_name, elapsed_time, char_per_second):
    print(f'Модель - {model_name}')
    print(f'Затраченное время: {elapsed_time} сек.')
    print(f'Скорость обработки: {char_per_second} символов/сек')

In [477]:
time_tests('gpt-4o-mini', elapsed_time_gpt4m, chars_sec_gpt4m)

Модель - gpt-4o-mini
Затраченное время: 141.01626014709473 сек.
Скорость обработки: 864.6166043037836 символов/сек


In [478]:
time_tests('gpt-3.5-turbo', elapsed_time_gpt3_5t, chars_sec_gpt3_5t)

Модель - gpt-3.5-turbo
Затраченное время: 166.5459406375885 сек.
Скорость обработки: 732.0802868760056 символов/сек


In [479]:
time_tests('vllm-llama3.1-70b-4q', elapsed_time_llama, chars_sec_llama)

Модель - vllm-llama3.1-70b-4q
Затраченное время: 215.12985396385193 сек.
Скорость обработки: 566.7507217314754 символов/сек


### recall, presicion, f1

Искать recall, presicion, f1 будем как Macro-averaged метрики, т.е. искать метрики для каждой пары (список правильных имен и предсказанных имен) и выводит средние значения.

In [480]:
#Импортируем функцию mean из модуля statistics для вычисления средних значений по спискам.
from statistics import mean 

def metrics_eval(names_pred, names_true, model_name):
    #Создаем три пустых списка для хранения значений Precision, Recall и F1-score для каждого текста в наборе данных.
    precision_list = []
    recall_list = []
    f1_list = []
    
    #Проходимся по каждому тексту в списке names_true, который содержит правильные сущности (персоны), 
    #и одновременно по соответствующим предсказаниям в списке names. Здесь предполагается, 
    #что names_true и names_pred — это списки списков, где каждый элемент — это список имен для конкретного текста.
    for i in range(len(names_true)):
    
        #Преобразуем списки имен в множества (set_true и set_pred), чтобы удобно сравнивать элементы и находить пересечения.
        set_true = set(names_true[i])  # список правильных ответов
        set_pred = set(names_pred[i])  # список предсказанных сущностей
    
        #Вычисление TP, FP и FN
        tp = len(set_true & set_pred)  # Пересечение
        fp = len(set_pred - set_true)  # Только в предсказанных
        fn = len(set_true - set_pred)  # Только в истинных
        
        #Вычисление Precision, Recall и F1 Score
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
        #Для каждого текста добавляем вычисленные Precision, Recall и F1-score в соответствующие списки.
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
    
    #Вывод средних значений метрик
    precision_mean = f"Precision (mean): {round(mean(precision_list), 3)}"
    recall_mean = f"Recall (mean): {round(mean(recall_list), 3)}"
    f1_score_mean = f"F1-score (mean): {round(mean(f1_list), 3)}"

    #return precision_mean, recall_mean, f1_score_mean

    print(f"Model name: {model_name}")
    print(precision_mean)
    print(recall_mean)
    print(f1_score_mean)

## Результаты для gpt4-mini, gpt3_5-turbo и vllm-llama3.1-70b-4q (для 100 запросов). 
## Для исходных форм и стеммизированных считаем отдельно.

#### Без стемминга

In [493]:
#gpt4_mini - без стемминга
metrics_eval(names_gpt4m, texts_persons, 'gpt4_mini')

Model name: gpt4_mini
Precision (mean): 0.497
Recall (mean): 0.404
F1-score (mean): 0.438


In [494]:
#gpt3_5_turbo - без стемминга
metrics_eval(names_gpt3_5t, texts_persons, 'gpt3_5-turbo')

Model name: gpt3_5-turbo
Precision (mean): 0.401
Recall (mean): 0.342
F1-score (mean): 0.355


In [495]:
#vllm-llama3.1-70b-4q - без стемминга
metrics_eval(names_llama, texts_persons, 'vllm-llama3.1-70b-4q')

Model name: vllm-llama3.1-70b-4q
Precision (mean): 0.393
Recall (mean): 0.301
F1-score (mean): 0.327


#### После стемминга

In [496]:
#gpt4_mini - со стеммингом
metrics_eval(names_gpt4m_stem, texts_persons_stem, 'gpt4_mini (stemming)')

Model name: gpt4_mini (stemming)
Precision (mean): 0.588
Recall (mean): 0.496
F1-score (mean): 0.529


In [497]:
#gpt3_5_turbo - со стеммингом
metrics_eval(names_gpt3_5t_stem, texts_persons_stem, 'gpt3_5-turbo (stemming)')

Model name: gpt3_5-turbo (stemming)
Precision (mean): 0.507
Recall (mean): 0.439
F1-score (mean): 0.452


In [486]:
#vllm-llama3.1-70b-4q - со стеммингом
metrics_eval(names_llama_stem, texts_persons_stem, 'vllm-llama3.1-70b-4q (stemming)')

Model name: vllm-llama3.1-70b-4q (stemming)
Precision (mean): 0.487
Recall (mean): 0.394
F1-score (mean): 0.417


Готово! 

- Лучший результат (как до, так и после стемминга) показал модель gpt4-mini.
- gpt3_5-turbo показала схожий результат с vllm-llama3.1-70b-4q, однако оказалась немного точнее. 
- Применение стемиминга значительно повысило результаты по метрикам для всех моделей.

## Типовые ошибки

1. Модель предсказать только часть полного имени. Например, предсказывает "Иван", когда в оригинальном тексте "Иван Иванов".
2. Модель путает именованные сущности с другими словами, например, "Московский" может быть предсказано как имя, хотя в контексте оно может быть связано с географией или являться прилагательным.
3. Модель не распознает имя, если встречается в необычном контексте или нетипичном месте предложения,.
4. Модель путает сущности не-русскоязычного происхождения, написанные на русском.
5. Модель путает псевдоним (музыканта, например) с именем.
6. Модель может путать именованные сущности со прочими словами, начинающимися с заглавной буквы.

### Почему точность не 100% (далеко нет)

1. Модель может иметь сложности с распознаванием именованных сущностей, поскольку она не обучена специально для задачи NER.
2. Если в исходных данных или аннотациях есть ошибки или пропуски, это повлияет на результат предсказания.
3. Некоторые имена могут быть омонимами или иметь сложную структуру (например, восточные или сложные составные имена), что может приводить к ошибкам.
4. Некоторые сущности могут похожи на имена что может путать модель (например названия организаций, стран и т.п.)
5. Запрос по поиску сущностей требует дополнительной модификации для более точного ответа (Например, начали извлекаться "Из", если написать в запросе "из текста").